In [1]:
from datetime import datetime
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import TimeoutException



In [4]:
origin = "BWI"
destination = "NYC"
date ="07/16/2024"

In [3]:

input_day = datetime.strptime(date, "%m/%d/%Y").day
day = str(input_day)


print(day)

if destination == "NYC":
    destination = "JFK"

print(destination)
# Set up the WebDriver
driver = webdriver.Chrome()
# Navigate to the Expedia flights page
driver.get("https://www.expedia.com/Flights")

# Click on the One-way link
one_way_link = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, '//a[contains(@class, "uitk-tab-anchor") and contains(.,"One-way")]'))
    )
one_way_link.click()

# Click on the "Leaving from" button
origin_button = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, '//button[@aria-label="Leaving from"]'))
)
origin_button.click()

# Enter origin
origin_input = WebDriverWait(driver, 10).until(
    EC.visibility_of_element_located((By.XPATH, '//input[@placeholder="Leaving from"]'))
)
origin_input.send_keys(origin)
origin_input.send_keys(Keys.RETURN)

# Click on the "Going to" button
destination_button = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, '//button[@aria-label="Going to"]'))
)
destination_button.click()

# Enter destination
destination_input = WebDriverWait(driver, 10).until(
    EC.visibility_of_element_located((By.XPATH, '//input[@placeholder="Going to"]'))
)
destination_input.send_keys(destination)
destination_input.send_keys(Keys.RETURN)


16
JFK


In [5]:

# Click on the Date Picker button to open the Date Picker
date_picker_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, "date_form_field-btn")))
date_picker_button.click()

# Wait for the calendar to load
WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.CLASS_NAME, 'uitk-calendar')))


<selenium.webdriver.remote.webelement.WebElement (session="371d0e78299ad56b34a719d509aa8404", element="f.95302EE6EB24E173974152065DE29CDE.d.4A9CE3EBDF7361826231ACC11B73F061.e.94")>

In [6]:
date_object = datetime.strptime(date, "%m/%d/%Y")

# Format the datetime object as "Month Year"
formatted_date = date_object.strftime("%B %Y")
print(formatted_date)

July 2024


In [8]:
h2_element = driver.find_elements(By.CLASS_NAME,"uitk-date-picker-month-name")
print(len(h2_element))
print(h2_element[0].text)
print(h2_element[1].text)



while h2_element[0].text != formatted_date:
    next_button = driver.find_elements(By.XPATH, "//button[@data-stid='date-picker-paging']")[1]
    next_button.click()
    h2_element = driver.find_elements(By.CLASS_NAME,"uitk-date-picker-month-name")
    print(h2_element[0].text)
    print(h2_element[1].text)



2
May 2024
June 2024
June 2024
July 2024
July 2024
August 2024


In [9]:

day_path = '//button[@data-day="' + day + '"]'
# Click on the specific day
day_button = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, day_path))
)
day_button.click()


In [10]:

# Click on the "Done" button
done_button = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, '//button[@data-stid="apply-date-picker"]'))
)
done_button.click()

# Click on the search button
search_button = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, '//button[@id="search_button"]'))
)
search_button.click()
time.sleep(10)

# Wait for the flight listings to load
WebDriverWait(driver, 60).until(
    EC.visibility_of_element_located((By.XPATH, '//li[@data-test-id="offer-listing"]'))
)


<selenium.webdriver.remote.webelement.WebElement (session="371d0e78299ad56b34a719d509aa8404", element="f.95302EE6EB24E173974152065DE29CDE.d.617B3CAA697300C95796192EB47711B0.e.246")>

In [11]:
while True:
    try:
        # Wait for the button to be clickable
        button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//button[@name='showMoreButton']"))
        )
        
        # Click the button
        button.click()
        print("Button clicked successfully!")
        
        # Sleep for a short while to allow the page to load after clicking
        time.sleep(2)  # Adjust the sleep time as needed
        
    except TimeoutException:
        # If the button is not found or not clickable, break out of the loop
        print("Button no longer available or clickable. Stopping.")
        break


Button no longer available or clickable. Stopping.


In [12]:
all_listing_info = []

while True:
    # Find all flight listings
    listings = driver.find_elements(By.XPATH, '//li[@data-test-id="offer-listing"]')
    # Iterate through each listing
    for listing in listings:
        # Dictionary to store information for the current listing
        listing_info = {}

        # Find elements within the current listing
        duration = listing.find_element(By.CSS_SELECTOR, '[data-test-id="journey-duration"]').text
        price = listing.find_element(By.CSS_SELECTOR, '.uitk-lockup-price').text
        airline_info = listing.find_element(By.CSS_SELECTOR, '[data-test-id="flight-operated"]').text.split(' • ')[0]
        dept, arr = listing.find_element(By.CSS_SELECTOR, '[data-test-id="departure-time"]').text.split(' - ')    

        # link_button = listing.find_element(By.XPATH, './/button[@data-test-id="select-link"]')
        # link = link_button.get_attribute("onclick").split("window.open('")[1].split("')")[0]     

        # Store information in the dictionary
        # listing_info['Link'] = link
        listing_info['Mode'] = "Plane"
        listing_info['Dept'] = dept
        listing_info['Arr'] = arr
        listing_info['Duration'] = duration.split('(')[0].strip()
        listing_info['Price'] = price
        listing_info['Company'] = airline_info

        # Add the dictionary to the list
        all_listing_info.append(listing_info)

    # Scroll to load more listings
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(10)
    # Wait for some time to let new listings load
    WebDriverWait(driver, 10).until(
        EC.visibility_of_element_located((By.XPATH, '//li[@data-test-id="offer-listing"]'))
    )
    more_listings = driver.find_elements(By.XPATH, '//li[@data-test-id="offer-listing"]')
    print(len(more_listings))
    print(len(listings))
    if len(more_listings) == len(listings):
        break

#Print all flight listings
# for listing_info in all_listing_info:
#     print(listing_info)
#     print("\n")

print("done - expedia.com")

# Close the browser
time.sleep(30)
driver.quit()


22
22
done - expedia.com


In [13]:
print(all_listing_info)

[{'Mode': 'Plane', 'Dept': '5:00pm', 'Arr': '6:39pm', 'Duration': '1h 39m', 'Price': '$134', 'Company': 'Delta'}, {'Mode': 'Plane', 'Dept': '6:05am', 'Arr': '7:23am', 'Duration': '1h 18m', 'Price': '$134', 'Company': 'Delta'}, {'Mode': 'Plane', 'Dept': '11:34am', 'Arr': '9:59pm', 'Duration': '10h 25m', 'Price': '$277', 'Company': 'American Airlines'}, {'Mode': 'Plane', 'Dept': '10:25am', 'Arr': '7:00pm', 'Duration': '8h 35m', 'Price': '$297', 'Company': 'American Airlines'}, {'Mode': 'Plane', 'Dept': '1:33pm', 'Arr': '9:55pm', 'Duration': '8h 22m', 'Price': '$297', 'Company': 'American Airlines'}, {'Mode': 'Plane', 'Dept': '10:25am', 'Arr': '9:55pm', 'Duration': '11h 30m', 'Price': '$297', 'Company': 'American Airlines'}, {'Mode': 'Plane', 'Dept': '6:18am', 'Arr': '3:30pm', 'Duration': '9h 12m', 'Price': '$297', 'Company': 'American Airlines'}, {'Mode': 'Plane', 'Dept': '5:00am', 'Arr': '3:30pm', 'Duration': '10h 30m', 'Price': '$297', 'Company': 'American Airlines'}, {'Mode': 'Plane',